In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

import threading

In [4]:
import os

def display_images_with_delete_function(image_directory, interval=3):
    # 画像ファイルのリストを取得
    images = [f for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    image_path = [os.path.join(image_directory, image) for image in images]
    total_images = len(images)

    # 画像とファイル名、インデックスを表示するためのウィジェットを作成
    image_widget = widgets.Image(
        value=open(image_path[0], 'rb').read() if images else None,
        format='png',
        width=224,
        height=224,
    )
    filename_label = widgets.Label(value=f"{images[0]} - 1/{total_images}" if images else "No images available")

    # 再生・停止・戻る・削除のためのフラグと現在の画像インデックス
    is_playing = False
    current_image = 0

    # 画像を更新する関数
    def update_image():
        nonlocal current_image
        if images:
            with open(image_path[current_image], 'rb') as file:
                image_widget.value = file.read()
                filename_label.value = f"{images[current_image]} - {current_image + 1}/{total_images}"
        else:
            image_widget.value = None
            filename_label.value = "No images available"

    # 自動再生を行う関数
    def autoplay_images():
        nonlocal is_playing, current_image
        while is_playing and current_image < total_images:
            update_image()
            time.sleep(interval)
            current_image += 1
            if current_image >= total_images:
                current_image = 0

    # 再生ボタンのイベントハンドラ
    def on_play_clicked(b):
        nonlocal is_playing
        if not is_playing and images:
            is_playing = True
            threading.Thread(target=autoplay_images).start()

    # 停止ボタンのイベントハンドラ
    def on_stop_clicked(b):
        nonlocal is_playing
        is_playing = False

    # 戻るボタンのイベントハンドラ
    def on_back_clicked(b):
        nonlocal current_image
        if images:
            if current_image > 0:
                current_image -= 1
            else:
                current_image = total_images - 1
            update_image()

    # 削除ボタンのイベントハンドラ
    def on_delete_clicked(b):
        nonlocal current_image, total_images
        if images:
            # 削除処理
            os.remove(image_path[current_image])
            del images[current_image]
            del image_path[current_image]
            total_images -= 1
            if current_image >= total_images:
                current_image = total_images - 1
            update_image()

    # 再生・停止・戻る・削除ボタンの作成
    play_button = widgets.Button(description="Play")
    stop_button = widgets.Button(description="Stop")
    back_button = widgets.Button(description="Back")
    delete_button = widgets.Button(description="Delete")
    play_button.on_click(on_play_clicked)
    stop_button.on_click(on_stop_clicked)
    back_button.on_click(on_back_clicked)
    delete_button.on_click(on_delete_clicked)

    # ウィジェットを表示
    display(widgets.HBox([play_button, stop_button, back_button, delete_button]))
    display(filename_label)
    display(image_widget)

# 使用例
display_images_with_delete_function('../dataset/collect/', interval=3)

# ノートブック環境での実行のため、実際のディレクトリパスは指定できませんが、
# これをあなたのJupyter Notebookにコピーして、適切なパスで実行してください。


Label(value='sample0.jpg - 1/3')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

TraitError: The 'value' trait of an Image instance expected a memory view object, not the NoneType None.